In [1]:
from mpi4py import MPI

In [ ]:
#EX0: MPI elements
comm=MPI.COMM_WORLD
my_rank=comm.Get_rank() #Rank is the id of each machine
p=comm.Get_size() #Size is the total number of machines
PID=os.getpid()
print(f'rank: {rank} has id: {PID}')
# mpirun -n 4 python mpi_simple.py

In [ ]:
#EX0: Point-to-point communications
#comm.send(obj,dest,tag=0)
#comm.recv(source)
if rank==0:
    msg='Hello'
    comm.send(msg,dest=1)
else:
    s=comm.recv()
    print("received ",s)

In [ ]:
#EX1: send messages from other processors to 0th.
comm=MPI.COMM_WORLD
my_rank=comm.Get_rank()
p=comm.Get_size()
if my_rank!=0:
    message="Hello from "+str(my_rank)
    comm.send(message,dest=0)
else:
    for Id in range(1,p):
        message=comm.recv(source=Id)
        print("Process 0 receives message from process ",Id,":",message)

In [3]:
#Auxiliary functions to use later
def f(x):
    return x*x*x
def Trap(a,b,n,h):
    integral=(f(a)+f(b))/2.0
    x=a
    for i in range(1,int(n)):
        x+=h
        integral+=f(x)
    return integral*h

In [6]:
#EX2: Compute trapezoid area numerically using parallel computation
comm=MPI.COMM_WORLD
my_rank=comm.Get_rank()
p=comm.Get_size()
a=0.0
b=1.0
n=1024
dest=0
total=-1
h=(b-a)/n
local_n=n/p
local_a=a+my_rank*h*local_n
local_b=local_a+local_n*h
integral=Trap(local_a,local_b,local_n,h)
if my_rank==0:
    total=integral
    for Id in range(1,p):
        integral=comm.recv(source=Id)
        print("Process ",my_rank," receives partial result from ",Id,'\n')
        total+=integral
else:
    print("Process ",my_rank," sends partial result \n")
    comm.send(integral,dest=0)
if my_rank==0:
    print("With n=",n," trapezoid integral \n")
    print("from ",a," to ",b," is ",total)

With n= 1024  trapezoid integral 

from  0.0  to  1.0  is  0.2500002384185791


In [ ]:
#EX3.0: Broadcast
#Create some data in rank 0 node
if my_rank==0:
    data={'key1':[1,2,3,4],'key2':('abc','deer')}
else:
    data=None
#Broadcast data from root 0 to all
comm.broadcast(data,root=0)
#Append the rank to the data
data['key1'].append(my_rank)
#Print the resulting data
print(f'rank={my_rank}, data={data}')

In [2]:
#EX3.1: Broadcast
comm=MPI.COMM_WORLD
my_rank=comm.Get_rank()
p=comm.Get_size()
def Get_data(my_rank,p,comm):
    a=None
    b=None
    n=None
    if my_rank==0:
        print("Rank: ",my_rank," enter a,b,n \n")
        a=float(input("enter a \n"))
        b=float(input("enter b \n"))
        n=int(input("enter n \n"))
        print("Ready for broadcast")
    a=comm.bcast(a)
    b=comm.bcast(b)
    n=comm.bcast(n)
    return a,b,n
Get_data(my_rank,p,comm)

Rank:  0  enter a,b,n 

enter a 
0
enter b 
1
enter n 
1024
Ready for broadcast


(0.0, 1.0, 1024)

In [6]:
#EX4: Trapezoid with Reduce
comm=MPI.COMM_WORLD
my_rank=comm.Get_rank()
p=comm.Get_size()
a,b,n=Get_data(my_rank,p,comm)
dest=0
#Initialization
total=-1.0
h=(b-a)/n
local_n=n/p
local_a=a+my_rank*h*local_n
local_b=local_a+local_n*h
integral=Trap(local_a,local_b,local_n,h)
#Add up the partial integrals calculated from each processor
total=comm.reduce(integral)
if my_rank==0:
    print("With n=",n,"trapezoid integral from \n")
    print("a=",a," to b=",b," is ",total)
MPI.Finalize

Rank:  0  enter a,b,n 

enter a 
0
enter b 
1
enter n 
1024
Ready for broadcast
With n= 1024 trapezoid integral from 

a= 0.0  to b= 1.0  is  0.2500002384185791


<function mpi4py.MPI.Finalize>

In [10]:
#Scatter and Gather
import numpy as np
comm=MPI.COMM_WORLD
my_rank=comm.Get_rank()
size=comm.Get_size()
#Generate a large array of numbers on rank 0 node
n=1000000
data=None
if my_rank==0:
    data=np.random.normal(loc=10,scale=5,size=n)
#Initialize an empty array to receive data
partial=np.empty(int(n/size),dtype='d')
#Send data to other workers
comm.Scatter(data,partial,root=0)
#Prepare reduced array to gather the scattered data
reduced=None
if my_rank==0:
    reduced=np.empty(size,dtype='d')
#Gather the data
comm.Gather(np.average(partial),reduced,root=0)
#Full average of set of averages
if my_rank==0:
    print("Full Average: ",np.average(reduced))

Full Average:  9.997483635705281
